In [15]:
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')
import numpy as np
from sklearn.cluster import KMeans
from sklearn import preprocessing,cross_validation
import pandas as pd

df = pd.read_excel("titanic.xls")
print(df.head())

   pclass  survived                                             name     sex  \
0       1         1                    Allen, Miss. Elisabeth Walton  female   
1       1         1                   Allison, Master. Hudson Trevor    male   
2       1         0                     Allison, Miss. Helen Loraine  female   
3       1         0             Allison, Mr. Hudson Joshua Creighton    male   
4       1         0  Allison, Mrs. Hudson J C (Bessie Waldo Daniels)  female   

       age  sibsp  parch  ticket      fare    cabin embarked boat  body  \
0  29.0000      0      0   24160  211.3375       B5        S    2   NaN   
1   0.9167      1      2  113781  151.5500  C22 C26        S   11   NaN   
2   2.0000      1      2  113781  151.5500  C22 C26        S  NaN   NaN   
3  30.0000      1      2  113781  151.5500  C22 C26        S  NaN   135   
4  25.0000      1      2  113781  151.5500  C22 C26        S  NaN   NaN   

                         home.dest  
0                     St Louis,

In [16]:
# convert first into a numerical data -: sex ----> numerical data 

df.drop(['body','name'],1,inplace=True)
df.head()
df.fillna(0,inplace=True) #useful for filling empty cell in the data 

In [17]:
#Hanling non numeric data 

def handle_non_numerical_data(df):
    columns = df.columns.values
    
    for column in columns:
        text_digit_vals = {}
        
        def convert_to_int(val):
            return text_digit_vals[val]
        
        if df[column].dtype != np.int64 and df[column].dtype != np.float64:
            column_contents = df[column].values.tolist()
            unique_elements = set(column_contents)
            x=0
            for unique in unique_elements:
                if unique not in text_digit_vals:
                    text_digit_vals[unique] = x
                    x+=1
                    
            df[column] = list(map(convert_to_int,df[column]))
            
    return df

df = handle_non_numerical_data(df)
print(df.head())
                
df.drop(['ticket'],1,inplace=True)   

   pclass  survived  sex      age  sibsp  parch  ticket      fare  cabin  \
0       1         1    1  29.0000      0      0     740  211.3375    161   
1       1         1    0   0.9167      1      2     495  151.5500     66   
2       1         0    1   2.0000      1      2     495  151.5500     66   
3       1         0    0  30.0000      1      2     495  151.5500     66   
4       1         0    1  25.0000      1      2     495  151.5500     66   

   embarked  boat  home.dest  
0         1     2         52  
1         1     9        336  
2         1     0        336  
3         1     0        336  
4         1     0        336  


In [18]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')
import numpy as np

colors = 10*["g","r","c","b","k"]

class K_Means:
    
    def __init__(self,k=2,tol=0.0001,max_iter=300):
        self.k = k
        self.tol = tol
        self.max_iter = max_iter
        
    def fit(self,data):
        self.centroids = {}
        
        for i in range(self.k):
            self.centroids[i] = data[i]
            
        for i in range(self.max_iter):
            self.classifications ={}
            
            for i in range(self.k):
                self.classifications[i] = []
             
            # for each of the featureset we are assigning it to a particular cluster by minising the distance of the particuaar
            # centroid of the corresponding cluster
            
            for featureset in data:
                distances = [np.linalg.norm(featureset-self.centroids[centroid]) for centroid in self.centroids]
                classification = distances.index(min(distances))
                self.classifications[classification].append(featureset)
                
            prev_centroids = dict(self.centroids)
            
            ##  ressigning the centroid of the cluster by recalculating the centroid of each cluster . After that we will again reclassify
            
            for classification in self.classifications:
                self.centroids[classification] = np.average(self.classifications[classification],axis=0)
                
            optimized = True
            
            for c in self.centroids:
                original_centroid = prev_centroids[c]
                current_centroid = self.centroids[c]
            
            if np.sum((current_centroid - original_centroid)/original_centroid *100.0) > self.tol :
                print(np.sum((current_centroid - original_centroid)/original_centroid *100.0))
                optimized = False
                
            if optimized:
                break

    def predict(self,data):
        distances = [np.linalg.norm(data-self.centroids[centroid]) for centroid in self.centroids]
        classification = distances.index(min(distances))
        return classification
    

In [19]:
X = np.array(df.drop(['survived'],1).astype(float))
#scaling the data 
X = preprocessing.scale(X)
y = np.array(df['survived'])
print(y)

clf = KMeans(n_clusters=2)
clf.fit(X)

[1 1 0 ..., 0 0 0]


/root/anaconda3/lib/python3.5/site-packages/sklearn/base.py:175: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  args, varargs, kw, default = inspect.getargspec(init)


KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=2, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)

In [20]:
correct = 0
for i in range(len(X)):
    predict_me = np.array(X[i].astype(float))
    predict_me = predict_me.reshape(-1,len(predict_me))
    prediction = clf.predict(predict_me)
    if prediction[0] == y[i]:
        correct += 1
        
print(correct/len(X))

0.7288006111535523
